**#Introduction**
**References**

 I)KazemiCVPR14
 
 II)Smile Detection in the Wild Based on Transfer Learning

**Rudmehr Aghakhani StudentNO: 4003663002** 

**#1**

In [33]:
import cv2
import mediapipe as mp
import os
import numpy as np

# Paths to folders
input_folder = 'nonsmile_face_detection_first'  # Folder containing 'smile' and 'nonsmile' folders
# output_folder_smile = 'smiled_face_detection_first_problematic_2_without_fault_0'
output_folder_nonsmile = 'nonsmile_face_detection_first_without_fault'

# Create output directories if they don't exist
# os.makedirs(output_folder_smile, exist_ok=True)
os.makedirs(output_folder_nonsmile, exist_ok=True)

# Initialize Mediapipe face mesh detector
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

# Standard dimensions for resized faces
standard_face_dim = (200, 200)  # Adjust as per requirement

# Helper function for alignment
def align_face(image, left_eye, right_eye, output_dim=(180, 192)):
    # Calculate the center of the eyes and angle for alignment
    eye_center = (left_eye + right_eye) /2.0
    dy = right_eye[1] - left_eye[1]
    dx = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dy, dx))
    
    # Calculate scale
    desired_distance = 70  # Distance between eyes in the aligned image
    current_distance = np.sqrt(dx ** 2 + dy ** 2)
    scale = desired_distance / current_distance
    
    # Rotation matrix for alignment
    rotation_matrix = cv2.getRotationMatrix2D(tuple(eye_center), angle, scale)
    
    # Apply the affine transformation
    aligned_face = cv2.warpAffine(image, rotation_matrix, output_dim)
    
    return aligned_face

# Process each folder ("smile" and "nonsmile")
for category in ["non_smile"]:
    input_path = os.path.join(input_folder, category)
    output_path = output_folder_smile if category == "smile" else output_folder_nonsmile

    # Loop through all images in the category folder
    for filename in os.listdir(input_path):
        img_path = os.path.join(input_path, filename)
        image = cv2.imread(img_path)

        if image is not None:
            # Resize image to improve consistency
            max_dim = 2100
            height, width = image.shape[:2]
            if max(height, width) > max_dim:
                scaling_factor = max_dim / float(max(height, width))
                image = cv2.resize(image, (int(width * scaling_factor), int(height * scaling_factor)))

            # Convert to RGB (required for Mediapipe)
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Detect facial landmarks
            result = face_mesh.process(rgb_image)
            if result.multi_face_landmarks:
                for landmarks in result.multi_face_landmarks:
                    # Extract eye positions
                    left_eye = np.array([landmarks.landmark[33].x * width, landmarks.landmark[33].y * height])
                    right_eye = np.array([landmarks.landmark[263].x * width, landmarks.landmark[263].y * height])

                    # Align the face based on eye positions
                    aligned_face = align_face(image, left_eye, right_eye, output_dim=standard_face_dim)
                    
                    # Save the aligned face
                    face_filename = f"{os.path.splitext(filename)[0]}_aligned_face.jpg"
                    cv2.imwrite(os.path.join(output_path, face_filename), aligned_face)
            else:
                print(filename)
        else:
            print(f"Could not open image: {img_path}")



file2188_aligned_face.jpg
file2345_aligned_face.jpg
file2365_aligned_face.jpg
file2395_aligned_face.jpg
file2527_aligned_face.jpg
file2576_aligned_face.jpg
file2586_aligned_face.jpg
file2594_aligned_face.jpg
file2664_aligned_face.jpg
file2669_face_1.jpg
file2757_aligned_face.jpg
file2853_aligned_face.jpg
file2932_aligned_face.jpg
file3064_aligned_face.jpg
file3155_aligned_face.jpg
file3369_face_0.jpg
file3383_aligned_face.jpg
file3389_aligned_face.jpg
file3412_aligned_face.jpg
file3582_aligned_face.jpg
file3608_aligned_face.jpg
file3670_aligned_face.jpg
file3769_aligned_face.jpg
file3786_aligned_face.jpg
file3787_aligned_face.jpg
file3824_aligned_face.jpg
file3838_aligned_face.jpg
file3862_aligned_face.jpg
file3892_aligned_face.jpg
file3924_aligned_face.jpg
file3975_aligned_face.jpg


**#2**

In [49]:
import cv2
import os

# Paths to folders
input_folder = 'problematic_3_non_smile'  # Folder containing 'smile' and 'nonsmile' folders
output_folder_smile = 'nonsmile_face_detection_non_smile_problematic_3'
output_folder_nonsmile = 'nonsmile_face_detection_non_smile_problematic_3'

# Create output directories if they don't exist
# os.makedirs(output_folder_smile, exist_ok=True)
os.makedirs(output_folder_nonsmile, exist_ok=True)

# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Minimum face size (adjust based on expected face sizes)
min_face_size = (30, 30)  # Width, Height (pixels)

# Process each folder ("smile" and "nonsmile")
for category in ["non_smile"]:
    input_path = os.path.join(input_folder, category)
    output_path = output_folder_smile if category == "smile" else output_folder_nonsmile

    # Loop through all images in the category folder
    for filename in os.listdir(input_path):
        img_path = os.path.join(input_path, filename)
        image = cv2.imread(img_path)
        
        if image is not None:
            
            # Resize image for consistency (if images vary greatly in size)
            max_dim = 800  # Resize to a max dimension of 800 (maintain aspect ratio)
            height, width = image.shape[:2]
            if max(height, width) > max_dim:
                scaling_factor = max_dim / float(max(height, width))
                image = cv2.resize(image, (int(width * scaling_factor), int(height * scaling_factor)))

            # Convert to grayscale for face detection
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Detect faces with specified minimum size
            faces = face_cascade.detectMultiScale(
                gray, scaleFactor=1.001, minNeighbors=5, minSize=min_face_size
            )
            if len(faces) == 0:
                print(filename)
            # Save each detected face to the respective output folder
            for i, (x, y, w, h) in enumerate(faces):
                # Filter out detections that are too wide or too tall (likely not a face)
                aspect_ratio = w / h
                if 0.75 < aspect_ratio < 1.3:  # Accept only nearly square faces
                    face = image[y:y+h, x:x+w]
                    face_filename = f"{os.path.splitext(filename)[0]}_face_{i}.jpg"
                    print(img_path)
                    # Optionally resize cropped face to a standard size
                    face_resized = cv2.resize(face, (150, 150))  # Resize to 150x150 for uniformity
                    
                    cv2.imwrite(os.path.join(output_path, face_filename), face_resized)
                    print(img_path)
                else:
                    print(f"Skipped non-face detection in {filename}")

        else:
            print(f"Could not open image: {img_path}")


problematic_3_non_smile\non_smile\file2188.jpg
problematic_3_non_smile\non_smile\file2188.jpg
problematic_3_non_smile\non_smile\file2188.jpg
problematic_3_non_smile\non_smile\file2188.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2345.jpg
problematic_3_non_smile\non_smile\file2365.jpg
problematic_3_non_smile\non_smile\file2365.jpg
problematic_3_non_smile\non_smile\file2365.jpg
problematic_3_non_smile\non_smile\file2365.jpg
problematic_3_non_smile\non_smile\file2365.jpg
problematic_3

**#3**

In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np

# Paths
input_folder = 'E:\\workspace\\Final_detection\\non_smile'
output_folder = 'non_smiled_image_processed11'

# Create the output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Initialize Mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

# Desired dimensions
output_dim = (224, 224)

# Helper function for alignment
def align_face(image, left_eye, right_eye, output_dim=(224, 224)):
    # Calculate the center of the eyes and angle for alignment
    eye_center = (left_eye + right_eye) / 2.0
    dy = right_eye[1] - left_eye[1]
    dx = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dy, dx))
    
    # Calculate scale
    desired_distance = 112  # Half of the output width for symmetry
    current_distance = np.sqrt(dx ** 2 + dy ** 2)
    scale = desired_distance / current_distance
    
    # Rotation matrix for alignment
    rotation_matrix = cv2.getRotationMatrix2D(tuple(eye_center), angle, scale)
    
    # Apply the affine transformation to align face
    aligned_face = cv2.warpAffine(image, rotation_matrix, (output_dim[0], output_dim[1]))
    
    return aligned_face

# Process each image in the input folder
for filename in os.listdir(input_folder):
    img_path = os.path.join(input_folder, filename)
    image = cv2.imread(img_path)

    if image is not None:
        height, width = image.shape[:2]

        # Convert to RGB for Mediapipe
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Detect facial landmarks
        result = face_mesh.process(rgb_image)
        
        if result.multi_face_landmarks:
            for landmarks in result.multi_face_landmarks:
                # Extract eye positions for alignment
                left_eye = np.array([landmarks.landmark[33].x * width, landmarks.landmark[33].y * height])
                right_eye = np.array([landmarks.landmark[263].x * width, landmarks.landmark[263].y * height])
                
                # Align and resize the face
                aligned_face = align_face(image, left_eye, right_eye, output_dim=output_dim)
                
                # Save the aligned and resized face
                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, aligned_face)
        else:
            print(f"No face landmarks detected in {filename}")

    else:
        print(f"Could not open image: {filename}")

print("All images aligned and resized successfully!")


No face landmarks detected in file2365_face_0.jpg
No face landmarks detected in file2669_face_1.jpg
No face landmarks detected in file3369_face_0.jpg
No face landmarks detected in file3838_face_1.jpg
No face landmarks detected in file3932_aligned_face_aligned_face.jpg
All images aligned and resized successfully!


**#4**

In [2]:
import os
import cv2

def resize_images_in_folder(input_folder, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):  # Add more formats if needed
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Read the image
            img = cv2.imread(input_path)
            if img is not None:
                # Resize the image
                resized_img = cv2.resize(img, (224, 224))
                # Save the resized image
                cv2.imwrite(output_path, resized_img)
                print(f'Resized {filename} and saved to {output_folder}')
            else:
                print(f'Failed to read {filename}')

# Specify your input and output folders
input_folder = 'resized_images'
output_folder = 'resized_images_224_224'

resize_images_in_folder(input_folder, output_folder)


Resized file2669_face_1.jpg and saved to resized_images_224_224
Resized file3838_face_1.jpg and saved to resized_images_224_224
Resized file3932_aligned_face_aligned_face.jpg and saved to resized_images_224_224


**#5**

In [ ]:
import os
import re

# Paths to the folders
folder1 = 'nonsmile_face_detection_first_without_fault'  # Folder with 1838 images
folder2 = 'E:\\workspace\\kaggle-genki4k\\non_smile'  # Folder with 1832 images

# Regular expression to extract the file number from filenames starting with 'file'
file_pattern = re.compile(r"file(\d+)")

# Function to extract file numbers from filenames in a folder
def get_file_numbers(folder_path):
    file_numbers = set()
    for filename in os.listdir(folder_path):
        match = file_pattern.match(filename)
        if match:
            file_number = int(match.group(1))
            file_numbers.add(file_number)
    return file_numbers

# Extract file numbers from each folder
file_numbers_folder1 = get_file_numbers(folder1)
file_numbers_folder2 = get_file_numbers(folder2)

# Find missing file numbers in each folder
missing_in_folder2 = file_numbers_folder1 - file_numbers_folder2
missing_in_folder1 = file_numbers_folder2 - file_numbers_folder1

# Output the missing files
if missing_in_folder2:
    print("Files in folder1 but missing in folder2:")
    for file_number in sorted(missing_in_folder2):
        print(f"file{file_number}")
else:
    print("No files are missing in folder2 from folder1.")

if missing_in_folder1:
    print("\nFiles in folder2 but missing in folder1:")
    for file_number in sorted(missing_in_folder1):
        print(f"file{file_number}")
else:
    print("\nNo files are missing in folder1 from folder2.")


No files are missing in folder2 from folder1.

Files in folder2 but missing in folder1:
file2188
file2345
file2365
file2576
file2889
file3608
file3838


**#6**

In [ ]:
import os
import re

# Path to the folder
folder_path = 'nonsmile_face_detection_first_without_fault'

# Regular expression to extract the file number from filenames starting with 'filename'
file_pattern = re.compile(r"filename(\d+)")

# Track seen numbers and duplicates
seen_numbers = set()
duplicate_files = []

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    match = file_pattern.match(filename)
    if match:
        file_number = int(match.group(1))
        
        # Check if the file number is already seen
        if file_number in seen_numbers:
            duplicate_files.append(filename)
        else:
            seen_numbers.add(file_number)

# Output the results
if duplicate_files:
    print("Duplicate files found:")
    for duplicate in duplicate_files:
        print(duplicate)
else:
    print("No duplicate files found.")


No duplicate files found.


**#7***

In [3]:
import cv2
import os

# Paths
input_folder = 'non_smiled_image_processed11'       # Path to the folder containing original images
output_folder = 'non_smiled_image_processed_denoised'     # Path to save denoised images

# Create the output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Process each image in the input folder
for filename in os.listdir(input_folder):
    img_path = os.path.join(input_folder, filename)
    image = cv2.imread(img_path)

    if image is not None:
        # Apply Non-Local Means Denoising
        denoised_image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

        # Save the denoised image to the output folder
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, denoised_image)
    else:
        print(f"Could not open image: {filename}")

print("Noise reduction applied to all images successfully!")


Noise reduction applied to all images successfully!
